In [1]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [2]:
import deep_learning

2024-05-27 11:10:00.593402: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-27 11:10:00.623511: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 11:10:01.117328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
with open("full_data.pic", "rb") as r:
    seeds_total, traces_total, key, round_perms_labels, copy_perms_labels, masks_labels, rws_perms_labels, rws_masks_labels = pic.load(r)

In [4]:
X_total, y_total = deep_learning.prepare_data_dl(False, traces_total, round_perms_labels, copy_perms_labels, masks_labels, rws_perms_labels, rws_masks_labels)

X_profiling, X_extraction = train_test_split(X_total, test_size=100_000, random_state=0)
y_profiling = {}
y_extraction = {}
for label in y_total.keys():
    train, test = train_test_split(y_total[label], test_size=100_000, random_state=0)
    y_profiling[label] = train
    y_extraction[label] = test

In [5]:
try:
    deep_learning.check_file_exists("./resnet_models/resnet_300000_smaller_windows.keras")
    from tensorflow.keras.models import load_model
    resnet = load_model("./resnet_models/resnet_300000_smaller_windows.keras")
except ValueError:
    resnet = deep_learning.ResNetSCA(epochs=100, dataset_size=X_profiling.shape[0], batch_size=32, rws=False)
    

2024-05-27 11:10:14.280975: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 11:10:14.281235: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 11:10:14.371747: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:41:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 11:10:14.371796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-27 11:10:14.371818: I external/local_xla/xla/stream_executor

In [6]:
history = resnet.train_model(X_profiling, y_profiling, "./resnet_models/resnet_300000_smaller_windows.keras", patience=10, validation_split=0.1)

Epoch 1/100


I0000 00:00:1716801029.019360   66685 service.cc:145] XLA service 0x7f1eac0106f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716801029.020176   66685 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-05-27 11:10:29.333975: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-27 11:10:30.376078: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-05-27 11:10:34.750074: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 10.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-27 11:10:35.822804: W external/local_tsl/tsl/framework/bfc_allocator.cc:296

   1/8301 ━━━━━━━━━━━━━━━━━━━━ 47:56:34 21s/step - block_perm_0_output_accuracy: 0.0625 - loss: 47.3919 - mask_0_0_output_accuracy: 0.0000e+00 - mask_0_1_output_accuracy: 0.0000e+00 - mask_0_2_output_accuracy: 0.0000e+00 - mask_0_3_output_accuracy: 0.0000e+00 - mask_0_4_output_accuracy: 0.0000e+00 - mask_0_5_output_accuracy: 0.0000e+00 - mask_0_6_output_accuracy: 0.0000e+00 - round_perm_output_accuracy: 0.1250

I0000 00:00:1716801040.020733   66685 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8300/8301 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - block_perm_0_output_accuracy: 0.2629 - loss: 44.0200 - mask_0_0_output_accuracy: 0.0038 - mask_0_1_output_accuracy: 0.0039 - mask_0_2_output_accuracy: 0.0038 - mask_0_3_output_accuracy: 0.0039 - mask_0_4_output_accuracy: 0.0043 - mask_0_5_output_accuracy: 0.0038 - mask_0_6_output_accuracy: 0.0039 - round_perm_output_accuracy: 0.0798

2024-05-27 11:14:56.326180: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-27 11:14:56.867977: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-27 11:14:56.937833: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 32.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-05-27 11:14:57.008059: W external/local_tsl/tsl/framework/bfc

8301/8301 ━━━━━━━━━━━━━━━━━━━━ 291s 33ms/step - block_perm_0_output_accuracy: 0.2630 - loss: 44.0198 - mask_0_0_output_accuracy: 0.0038 - mask_0_1_output_accuracy: 0.0039 - mask_0_2_output_accuracy: 0.0038 - mask_0_3_output_accuracy: 0.0039 - mask_0_4_output_accuracy: 0.0043 - mask_0_5_output_accuracy: 0.0038 - mask_0_6_output_accuracy: 0.0039 - round_perm_output_accuracy: 0.0798 - val_block_perm_0_output_accuracy: 0.5184 - val_loss: 44.8076 - val_mask_0_0_output_accuracy: 0.0050 - val_mask_0_1_output_accuracy: 0.0037 - val_mask_0_2_output_accuracy: 0.0038 - val_mask_0_3_output_accuracy: 0.0053 - val_mask_0_4_output_accuracy: 0.0040 - val_mask_0_5_output_accuracy: 0.0044 - val_mask_0_6_output_accuracy: 0.0039 - val_round_perm_output_accuracy: 0.1055
Epoch 2/100
8301/8301 ━━━━━━━━━━━━━━━━━━━━ 262s 32ms/step - block_perm_0_output_accuracy: 0.8088 - loss: 41.6481 - mask_0_0_output_accuracy: 0.0067 - mask_0_1_output_accuracy: 0.0058 - mask_0_2_output_accuracy: 0.0049 - mask_0_3_output_accu

In [48]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical

y_predicted = load_model("./resnet_models/resnet_350000_52_epochs.keras").predict(X_extraction)
for i, k in enumerate(y_extraction.keys()):
    print(f"{k}: {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k])}/{y_extraction[k].shape[0]} = {np.count_nonzero(np.argmax(y_predicted[i], axis=1) == y_extraction[k]) / y_extraction[k].shape[0]}")

np.mean([categorical_crossentropy(to_categorical(y_extraction[k]), y_predicted[i]) for i, k in enumerate(y_extraction.keys())], axis=1)

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step
x_round: 24419/45133 = 0.5410453548401392
block_perm_0: 39134/45133 = 0.8670817362018922
mask_0_0: 354/45133 = 0.007843484811556953
mask_0_1: 365/45133 = 0.00808720891587087
mask_0_2: 305/45133 = 0.006757804710522234
mask_0_3: 302/45133 = 0.006691334500254802
mask_0_4: 332/45133 = 0.0073560366029291205
mask_0_5: 305/45133 = 0.006757804710522234
mask_0_6: 276/45133 = 0.006115259344603727


array([1.3536543, 0.3749589, 5.503599 , 5.5072737, 5.477    , 5.5458245,
       5.490437 , 5.568437 , 5.656007 ], dtype=float32)